In [ ]:
from experiments.attn.victory import *

In [ ]:
device = 'cuda'
D = 64
B = 8*1024
T = 500

worlds = Hex.initial(B, boardsize=7)

boardsize = worlds.boardsize
model = common.AttnModel(common.PosActions, boardsize, D).to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-2)

with tqdm(total=T) as pbar:
    for t, b in zip(range(T), batchgen(worlds, B)):
        targets = terminal_actions(b.worlds)
        outputs = model(b.worlds.obs)

        infs = torch.full_like(targets, -np.inf)
        loss = -outputs.reshape(B, -1).where(targets == 1., infs).max(-1).values.mean()

        opt.zero_grad()
        loss.backward()
        opt.step()

        pbar.update(1)
        pbar.set_description(f'{loss:.2f}')



In [ ]:
targets = terminal_actions(b.worlds)
outputs = model(b.worlds.obs)

infs = torch.full_like(targets, -np.inf)
loss = -outputs.reshape(B, -1).where(targets == 1., infs).max(-1).values

bad = loss.argsort().cpu().numpy()[::-1]

In [ ]:
plot(bad[10], b.worlds.obs, targets, outputs, model.attns)

In [ ]:
from rebar.recording import Encoder

i = 11
obs = b.worlds.obs[i].detach().cpu().numpy()
attn = model.attns[i].detach().cpu().numpy()
boardsize = int(attn.shape[1]**.5)
attn = attn.transpose(0, 2, 1).reshape(attn.shape[0], attn.shape[2], boardsize, boardsize)

rows, cols = attn.shape[:2]


with Encoder(fps=1) as enc:
    for r in range(rows+3):
        r = min(r, rows-1)
        fig, axes = plt.subplots(1, cols+1, squeeze=False)
        fig.set_size_inches(8*(cols+1), 8)

        plot_board(color_obs(obs), ax=axes[0, 0])
        for c in range(cols):
            colors = plt.cm.viridis(attn[r, c])
            plot_board(colors, ax=axes[0, c+1])
        enc(fig)
        plt.close(fig)
enc.notebook()